<img src="http://datapark.io/img/logo.png" alt="datapark" width="45%" align="right" border="4">

# IPython Parallel

In [ ]:
import numpy as np
from time import time

## LSM Monte Carlo for American Options

The following function implements the Least-Squares Monte Carlo (LSM) algorithm to value American options by Monte Carlo simulation.

In [ ]:
def optionValue(S0, vol, T, K=40, M=50, I=10 * 4096, r=0.06):
    import numpy as np
    np.random.seed(150000)  # fix the seed for every valuation
    dt = T / M  # time interval
    df = np.exp(-r * dt)  # discount factor per time time interval
    S = np.zeros((M + 1, I), dtype=np.float64)  # stock price matrix
    S[0, :] = S0  # intial values for stock price
    for t in range(1, M + 1):
        ran = np.random.standard_normal(I)
        S[t, :] = S[t - 1, :] * np.exp((r - vol ** 2 / 2) * dt
                        + vol * ran * np.sqrt(dt))
    h = np.maximum(K - S, 0)  # inner values for put option
    V = np.zeros_like(h)  # value matrix
    V[-1] = h[-1]
    for t in range(M - 1, 0, -1):
        rg = np.polyfit(S[t, :], V[t + 1, :] * df, 5)  # regression
        C = np.polyval(rg, S[t, :])  # evaluation of regression
        V[t, :] = np.where(h[t, :] > C, h[t, :],
                         V[t + 1, :] * df)  # exercise decision/optimization
    V0 = np.sum(V[1, :] * df) / I  # LSM estimator
    print ("S0 %4.1f|vol %4.2f|T %2.1f| Option Value %8.3f" % (S0, vol, T, V0))
    return V0

## Sequential Valuation

In [ ]:
def seqValue():
    optionValues = []
    for S0 in (36., 38., 40., 42., 44.):  # initial stock price values
        for vol in (0.2, 0.4):  # volatility values
            for T in (1.0, 2.0):  # times-to-maturity
                optionValues.append(optionValue(S0, vol, T))
    return optionValues

In [ ]:
t0 = time()
optionValues = seqValue()  # calculate all values
ts = time() - t0
print ("Sequential time in sec. %6.3f" % ts)

## Parallel Valuation

In [ ]:
from IPython.parallel import Client
cluster_profile = "default"
c = Client(profile=cluster_profile)
view = c.load_balanced_view()

In [ ]:
def parValue():
    optionValues = []
    for S in (36., 38., 40., 42., 44.):
        for vol in (0.2, 0.4):
            for T in (1.0, 2.0):
                value = view.apply_async(optionValue, S, vol, T)
                optionValues.append(value)
    c.wait(optionValues)
    return optionValues

In [ ]:
t0 = time()
optionValues = parValue()
tp = time() - t0
print ("Parallel time in sec. %6.3f" % tp)

In [ ]:
for result in optionValues:
    print (result.metadata['stdout'])

## Comparison

In [ ]:
%matplotlib nbagg
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
wi = 0.4
plt.bar((1.5 - wi/2, 2.5 - wi/2), (ts/ts, tp/ts), width=wi)
plt.xticks((1.5, 2.5), ('sequential', 'parallel'))
plt.grid(True), plt.ylim(0, 1.1), plt.xlim(0.75, 3.25)
plt.ylabel('relative performance (lower = better)')
plt.title('Sequential vs. Parallel LSM')

<img src="http://datapark.io/img/logo.png" alt="datpark" width="35%" align="right" border="0"><br>

<a href="http://datapark.io" target="_blank">datapark.io</a> | <a href="http://twitter.com/dataparkio" target="_blank">@datparkio</a> | <a href="mailto:team@datapark.io">team@datapark.io</a>
